In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
train = pd.read_csv('train.csv')
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [3]:
test = pd.read_csv('test.csv')
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [4]:
col_names = [f'var_{i}' for i in range(200)]
for col in tqdm(col_names):
    count = test[col].value_counts()
    uniques = count.index[count == 1]
    test[col + '_u'] = test[col].isin(uniques)
test["has_unique"] = test[[col + "_u" for col in col_names]].any(axis = 1)

In [5]:
test["has_unique"].sum()

100000

In [6]:
real_test = test.loc[test['has_unique'], ["ID_code"] + col_names]
fake_test = test.loc[~test['has_unique'], ["ID_code"] + col_names]
train_and_test = pd.concat([train, real_test], axis = 0)

In [7]:
for col in tqdm(col_names):
    count = train_and_test[col].value_counts().to_dict()
    train_and_test[col+"_unique"] = train_and_test[col].apply(
    lambda x: 1 if count[x] == 1 else 0).values
    fake_test[col+"_unique"] = 0

In [8]:
train_and_test

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190_unique,var_191_unique,var_192_unique,var_193_unique,var_194_unique,var_195_unique,var_196_unique,var_197_unique,var_198_unique,var_199_unique
0,train_0,0.0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,0,0,0,0,0,0,0,0,0,0
1,train_1,0.0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,0,0,0,0,0,0,0,0,0,0
2,train_2,0.0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,0,0,0,0,0,0,0,0,0,0
3,train_3,0.0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,0,0,0,0,0,0,0,0,0,0
4,train_4,0.0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,test_199986,NaN,19.2884,-2.8384,11.9149,6.6611,12.3112,12.9244,5.6492,16.0449,...,0,0,0,0,0,0,0,0,0,0
199993,test_199993,NaN,14.6764,-8.1066,7.1167,2.4138,10.3845,-11.9327,4.7563,16.0455,...,0,0,0,0,0,0,0,0,0,1
199995,test_199995,NaN,13.1678,1.0136,10.4333,6.7997,8.5974,-4.1641,4.8579,14.7625,...,1,0,0,0,1,0,0,0,0,1
199996,test_199996,NaN,9.7171,-9.1462,7.3443,9.1421,12.8936,3.0191,5.6888,18.8862,...,1,0,0,0,0,0,0,0,0,1


In [9]:
real_test = train_and_test[train_and_test["ID_code"].str.contains("test")].copy()
real_test.drop(['target'], axis =1, inplace = True)
train =  train_and_test[train_and_test["ID_code"].str.contains("train")].copy()

In [10]:
test = pd.concat([real_test, fake_test], axis = 0)
train.to_csv('new_train.csv', index = False)
test.to_csv('new_test.csv', index = False)